In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats

In [ ]:
summary = pd.read_csv("../Summary_cut.csv", index_col=0)
summary.cell_type = summary.cell_type.astype('category')
summary.sample_type = summary.sample_type.astype('category')
summary.sort_values(by="age", inplace=True)
summary.reset_index(inplace=True)
# neglect some duplicated colonies e.g. summary.colony_ID == "11_E07"
summary = summary.merge(
    summary[["donor_id", "age"]].groupby("donor_id").count().reset_index().rename(columns={"age": "cells"}),
    on="donor_id",
    validate="many_to_one",
    how="left"
)
summary.dtypes

In [ ]:
summary.describe()

In [ ]:
summary.cell_type.value_counts()

In [ ]:
summary.sample_type.value_counts()

In [ ]:
summary

In [ ]:
sns.regplot(data=summary, x="age", y="number_mutations", scatter_kws={"marker": "x"})

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(7, 3))
sns.histplot(
    data=summary, x="number_mutations", hue="donor_id", kde=True, binwidth=10, ax=ax,
    stat="percent",
)
sns.move_legend(ax, bbox_to_anchor=(1.01, 1), loc="upper left", frameon=False)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(7, 3))
sns.histplot(
    data=summary, x="number_mutations", hue="donor_id", kde=True, binwidth=10, ax=ax,
    stat="count",
)
sns.move_legend(ax, bbox_to_anchor=(1.01, 1), loc="upper left", frameon=False)
plt.show()

In [ ]:
for i in summary.donor_id.unique():
    fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(6, 4))
    sns.histplot(
        data=summary[summary.donor_id == i], x="number_mutations", hue="donor_id", kde=True, bins=50, ax=ax,
        stat="count"
    )
    plt.show()

## Entropy
Based on the code they [developped](https://github.com/emily-mitchell/normal_haematopoiesis/blob/23d221e8d125d78c1e8bcbe05d41d0f3594b0cfb/4_phylogeny_analysis/scripts/shannon_diversity.Rmd#L147), I think they define entropy as in [here](http://math.bu.edu/people/mkon/J6A.pdf) using the phylogenetic tree.
We just compute the entropy from the number of cells: we consider a class being the cells with the same number of mutations and compute the abbundance of those classes, that is the abbundance of cells with the same number of mutations.

In [ ]:
# count the number of cells with the same number of mutations
test = summary[["donor_id", "number_mutations", "age"]].groupby(["donor_id", "number_mutations"]).count().reset_index()
test.rename(columns={"age": "probs"}, inplace=True)
entropies = test[["donor_id", "probs"]].groupby("donor_id").value_counts().reset_index().groupby("donor_id").agg(stats.entropy).reset_index().rename(columns={"probs": "entropy"})
summary = summary.merge(
    entropies,
    how="left",
    on="donor_id",
    validate="many_to_one"
)

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(6, 4))
summary.drop_duplicates(subset=["entropy", "donor_id"]).plot(x="age", y="entropy", ax=ax, kind="scatter")
plt.show()